# Synthetic Radio Host Generator

In [ ]:
import os
import sys
import logging


logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    handlers=[logging.StreamHandler(sys.stdout)],
    force=True
)
logger = logging.getLogger(__name__)

REPO_URL = "https://github.com/Procrastinator02/synthetic_radio_host.git"
REPO_DIR = "/content/synthetic_radio_host"

if not os.path.exists(REPO_DIR):
    logger.info(f"Cloning repository from {REPO_URL}...")
    !git clone {REPO_URL} {REPO_DIR}
    logger.info("Repository cloned successfully")
else:
    logger.info("Repository already exists, updating...")
    !cd {REPO_DIR} && git pull


sys.path.insert(0, REPO_DIR)
logger.info(f"Added {REPO_DIR} to Python path")


In [ ]:
import os
import sys
import logging

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
logger = logging.getLogger(__name__)


if not os.path.exists("/content/.deps_installed"):
    logger.info("Installing dependencies from requirements.txt...")
    !pip install -q -r /content/synthetic_radio_host/requirements.txt
    open("/content/.deps_installed", "w").close()
    logger.info("Dependencies installed successfully")
else:
    logger.info("Dependencies already installed")


sys.path.insert(0, '/content/synthetic_radio_host/src')
logger.info("Added src/ to Python path")


Dependencies already installed — skipping Cell 1


# Setup and Configuration

In [ ]:

from radio_host_functions import (
    CONFIG,
    __version__,
    fetch_wikipedia_article,
    generate_script_prompt,
    generate_script,
    generate_audio_segments,
    combine_and_export_audio
)

from openai import OpenAI
from elevenlabs import Client as ElevenLabs
from google.colab import userdata, files


try:
    OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")
    ELEVENLABS_API_KEY = userdata.get("ELEVENLABS_API_KEY")
    
    if not OPENAI_API_KEY:
        raise ValueError("OPENAI_API_KEY not found in Colab secrets")
    if not ELEVENLABS_API_KEY:
        raise ValueError("ELEVENLABS_API_KEY not found in Colab secrets")
    
    logger.info("API keys loaded successfully")
except Exception as e:
    logger.error(f"Failed to load API keys: {e}")
    raise

In [ ]:
try:
    openai_client = OpenAI(api_key=OPENAI_API_KEY)
    eleven_client = ElevenLabs(api_key=ELEVENLABS_API_KEY)
    logger.info("API clients initialized successfully")
except Exception as e:
    logger.error(f"Failed to initialize API clients: {e}")
    raise

In [ ]:
WIKIPEDIA_TOPIC = "MS Dhoni"  
wiki_text = fetch_wikipedia_article(WIKIPEDIA_TOPIC)
logger.info(f"Fetched Wikipedia article: {WIKIPEDIA_TOPIC} ({len(wiki_text)} characters)")


In [ ]:
HINGLISH_PROMPT = generate_script_prompt(wiki_text)
logger.info("Generated script prompt")
script = generate_script(HINGLISH_PROMPT, openai_client)

In [ ]:
audio_segments = generate_audio_segments(script, eleven_client)
logger.info(f"Generated {len(audio_segments)} audio segments")


In [ ]:
combine_and_export_audio(
    audio_segments, 
    CONFIG["OUTPUT_FILENAME"],
    files_download=files.download
)
logger.info(f"Audio file generated: {CONFIG['OUTPUT_FILENAME']}")
